In [ ]:
# ===== Standard library =====
from pathlib import Path
from io import StringIO
import re

# ===== Third-party =====
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib.cm import Blues, Reds
from matplotlib.colors import to_rgb


# ----- File locations ---------------------------------------------------
ROOT      = Path(".") / "Experiment_004"
DIM_TXT   = ROOT / "Dimensional_Data.txt"
TEN_TXT   = ROOT / "Tensile_Data.txt"

In [3]:
def rgba(mat_color, alpha: float = 1.0) -> str:
    """
    Convert a Matplotlib colour (any valid *RGB tuple* or *hex string*)
    to a Plotly-compatible 'rgba(r,g,b,a)' string.

    Parameters
    ----------
    mat_color : tuple | str
        3-tuple of floats in 0-1 or any colour string accepted by
        ``matplotlib.colors.to_rgb`` (e.g. '#2a9d8f').
    alpha : float, default 1.0
        Alpha channel 0–1.

    Returns
    -------
    str
        CSS-style rgba(…) string.
    """
    r, g, b = (np.array(to_rgb(mat_color)) * 255).astype(int)
    return f"rgba({r},{g},{b},{alpha})"

In [4]:
class DimensionalData:
    """
    Parse a Dia-Stron dimensional *.txt* file and expose a
    *slot → mean-area (µm²)* mapping.

    Design goals
    ------------
    * Accept rows appended out of order (reruns).
    * Silently skip blank / malformed rows.
    * Avoid heavy deps (only NumPy / Pandas).
    """

    _AREA_RE = re.compile(r"cross.*area", re.I)      # locate area column
    _NUM_RE  = re.compile(r"^\d+(\.\d+)?$")          # basic numeric check

    def __init__(self, path: Path):
        lines = path.read_text().splitlines()
        hdr_idx = next(i for i, l in enumerate(lines) if l.startswith("Record"))
        header  = [c.strip() for c in lines[hdr_idx].split("\t")]

        try:
            area_idx = next(i for i, c in enumerate(header)
                            if self._AREA_RE.search(c))
        except StopIteration as exc:
            raise ValueError("Area column not found") from exc
        print(f"[DEBUG] Area column index = {area_idx}")

        slot_vals: dict[int, list[float]] = {}

        for row in lines[hdr_idx + 1 :]:
            if not row.strip():
                continue
            parts = row.split("\t")
            if len(parts) <= area_idx:
                continue

            rec, area = parts[0].strip(), parts[area_idx].strip()
            if not self._NUM_RE.match(area):
                continue

            try:
                slot = int(float(rec))
                area_val = float(area)
            except ValueError:
                continue

            slot_vals.setdefault(slot, []).append(area_val)

        if not slot_vals:
            raise ValueError("No numeric area entries parsed")

        self.map: dict[int, float] = {
            slot: float(np.mean(vals)) for slot, vals in slot_vals.items()
        }
        print(f"[DEBUG] Dimensional – slots parsed: {len(self.map)}")

In [7]:
# ── Cell 4  ────────────────────────────────────────────────────────────
class TensileTest:
    """
    Wrapper for Dia-Stron tensile .txt files.

    The file can contain either:
      • raw gram-force   (column 'gmf')
      • engineering MPa  (any header in MPa_HEADERS)

    Provides
    --------
    self.df        – cleaned full DataFrame
    per_slot()     – generator yielding (slot, df_slot)
    stress_strain  – convert df_slot → with strain + stress_Pa columns
    metrics()      – static helper for UTS, break values, Young’s modulus
    """

    GF_TO_N     = 0.00981
    MPa_HEADERS = {"mpa", "stress", "stress_mpa"}

    # ------------------------------------------------------------------ #
    # construction                                                       #
    # ------------------------------------------------------------------ #
    def __init__(self, path: Path):
        raw_lines = path.read_text().splitlines()

        header: list[str] | None = None
        body:   list[str] = []
        for ln in raw_lines:
            if ln.startswith("Record"):
                header = [c.strip() for c in ln.split("\t") if c.strip()]
            elif header and ln.strip():
                body.append(ln)

        if header is None:  # pragma: no cover
            raise ValueError("Header line starting with 'Record' not found")

        df = pd.read_csv(StringIO("\n".join(body)), sep="\t", names=header)

        # ---- normalise columns --------------------------------------------
        df.rename(columns=lambda c: c.strip(), inplace=True)
        if "% Strain" in df.columns:
            df.rename(columns={"% Strain": "Strain_pct"}, inplace=True)

        # ---- detect stress units ------------------------------------------
        col_lut  = {c.lower(): c for c in df.columns}
        mpa_cols = [orig for low, orig in col_lut.items()
                    if low in self.MPa_HEADERS]
        self.is_mpa = bool(mpa_cols)
        stress_col  = mpa_cols[0] if self.is_mpa else "gmf"
        print(f"[DEBUG] Tensile units: "
              f"{'MPa' if self.is_mpa else 'gmf'} (column '{stress_col}')")

        # ---- clean numeric data -------------------------------------------
        df.rename(columns={stress_col: "raw_stress"}, inplace=True)
        df["Record"]     = pd.to_numeric(df["Record"],     errors="coerce")
        df["Strain_pct"] = pd.to_numeric(df["Strain_pct"], errors="coerce")
        df["raw_stress"] = pd.to_numeric(df["raw_stress"], errors="coerce")

        self.df = (
            df.dropna(subset=["Record", "Strain_pct", "raw_stress"])
              .astype({"Record": int})
        )

    # ------------------------------------------------------------------ #
    # public helpers                                                     #
    # ------------------------------------------------------------------ #
    def per_slot(self):
        """Yield (slot_number, DataFrame) pairs in ascending slot order."""
        for slot, grp in self.df.groupby("Record", sort=True):
            yield slot, grp.reset_index(drop=True)

    def stress_strain(self, df_slot: pd.DataFrame, area_um2: float) -> pd.DataFrame:
        """Convert raw slot data → engineering strain + true stress (Pa)."""
        out = df_slot.copy()
        out["strain"] = out["Strain_pct"] / 100
        if self.is_mpa:
            out["stress_Pa"] = out["raw_stress"] * 1_000_000
        else:
            out["stress_Pa"] = (
                out["raw_stress"] * self.GF_TO_N / (area_um2 * 1e-12)
            )
        return out

    # ------------------------------------------------------------------ #
    # static metrics                                                     #
    # ------------------------------------------------------------------ #
    @staticmethod
    def metrics(df: pd.DataFrame) -> tuple[float, float, float, float]:
        """Return UTS, break stress, break strain, Young’s modulus."""
        s = df["stress_Pa"].to_numpy()
        e = df["strain"].to_numpy()

        idx = s.argmax()
        uts          = s[idx]              # Pa
        brk_stress   = s[idx]              # Pa (same as UTS in this file format)
        brk_strain   = e[idx] * 100        # %
        linear_mask  = e <= 0.002
        E = (
            np.polyfit(e[linear_mask], s[linear_mask], 1)[0]
            if linear_mask.sum() > 1 else np.nan
        )
        return uts / 1e6, brk_stress / 1e6, brk_strain, E / 1e9

    @staticmethod
    def yield_gradient(df: pd.DataFrame,
                       low_pct: float = 7.0,
                       high_pct: float = 16.0) -> float:
        """
        Slope (MPa / % strain) between `low_pct` and `high_pct` strain.
        """
        if high_pct <= low_pct:
            raise ValueError("high_pct must be greater than low_pct")

        e_pct = df["strain"].to_numpy() * 100
        s_pa  = df["stress_Pa"].to_numpy()
        if e_pct.size < 2:
            return np.nan
        # ensure ascending order before interpolation
        idx   = np.argsort(e_pct)
        e_pct = e_pct[idx]
        s_pa  = s_pa[idx]

        if e_pct.min() > low_pct or e_pct.max() < high_pct:
            return np.nan

        σ_low  = np.interp(low_pct,  e_pct, s_pa)
        σ_high = np.interp(high_pct, e_pct, s_pa)

        return ((σ_high - σ_low) / 1e6) / (high_pct - low_pct)

# ---- helper: post-gradient slope (MPa per % strain) -------------------
def post_gradient(proc_df: pd.DataFrame, delta_pct: float = 8.0) -> float:
    """
    Return slope MPa / % between break point and (break − Δε) point.
    """
    s = proc_df["stress_Pa"].to_numpy()
    e = proc_df["strain"].to_numpy()          # 0–1
    if len(s) < 3:
        return np.nan
    idx_break = s.argmax()
    brk_strain_pct = e[idx_break] * 100
    target_pct = brk_strain_pct - delta_pct
    if target_pct < 0:
        return np.nan

    anchor_idx = np.where(e * 100 <= target_pct)[0]
    if anchor_idx.size == 0:
        return np.nan
    anchor_idx = anchor_idx[-1]

    dσ_MPa = (s[idx_break] - s[anchor_idx]) / 1e6
    dε_pct = brk_strain_pct - e[anchor_idx] * 100
    return dσ_MPa / dε_pct if dε_pct else np.nan


In [8]:
import math
areas = DimensionalData(DIM_TXT).map
raw   = TensileTest(TEN_TXT)         # the class that now includes yield_gradient()

blues, reds = Blues(np.linspace(0.4, 0.9, 50)), Reds(np.linspace(0.4, 0.9, 50))
fig_overlay = go.Figure()

metric_rows, curve_rows = [], []
miss_area, empty_slot   = [], []
slots_plotted           = []

for slot, df_raw in raw.per_slot():
    if slot not in areas:
        miss_area.append(slot)
        continue

    proc = raw.stress_strain(df_raw, areas[slot])
    if proc.empty:
        empty_slot.append(slot)
        continue

    slots_plotted.append(slot)
    cond = "1-50" if slot <= 50 else "51-100"

    # --- store every curve point (for optional export / separate visual) ---
    curve_rows += [
        {"Slot": slot, "Condition": cond,
         "Strain": st, "Stress_MPa": sp / 1e6}
        for st, sp in zip(proc["strain"], proc["stress_Pa"])
    ]

    # --- overlay plot -------------------------------------------------------
    col = (rgba(blues[slot - 1]) if 1 <= slot <= 50
           else rgba(reds[slot - 51]) if 51 <= slot <= 100
           else "grey")
    fig_overlay.add_trace(go.Scattergl(
        x=proc["strain"], y=proc["stress_Pa"] / 1e6,
        mode="lines",
        line=dict(color=col, width=1),
        name=f"Slot {slot}",
        hovertemplate=f"Slot {slot}<br>Strain=%{{x:.3f}}<br>Stress=%{{y:.2f}} MPa"
    ))

    # --- per-slot metrics ---------------------------------------------------
    uts, bs, bp, E = TensileTest.metrics(proc)         
    grad_post = post_gradient(proc, delta_pct=8.0)     
    grad_yld  = TensileTest.yield_gradient(proc)      

    metric_rows.append({
        "Slot": slot,
        "Condition": cond,
        "UTS_MPa": uts,
        "Break_Stress_MPa": bs,
        "Break_Strain_%": bp,
        "Youngs_Modulus_GPa": E,
        "Post_Gradient_MPa_perc": grad_post,
        "Yield_Gradient_MPa_perc": grad_yld,           
    })

# ---- tidy DataFrames ---------------------------------------------------
summary_df = pd.DataFrame(metric_rows).set_index("Slot")
curves_df  = pd.DataFrame(curve_rows)

# ── Overlay buttons (use real slots) -----------------------------------
N       = len(fig_overlay.data)
vis_all = [True] * N
vis1    = [s <= 50 for s in slots_plotted]
vis2    = [s >= 51 for s in slots_plotted]

fig_overlay.update_layout(
    title="Experiment 002 – stress–strain curves",
    xaxis_title="Strain",
    yaxis_title="Stress (MPa)",
    template="simple_white",
    height=750,
    legend=dict(title="Toggle curves",
                y=1, x=1.02, orientation="v", borderwidth=1),
    updatemenus=[dict(
        type="buttons",
        direction="right",
        x=0.02, y=1.08, showactive=True,
        buttons=[
            dict(label="Show All",    method="update",
                 args=[{"visible": vis_all}]),
            dict(label="Slots 1–50",  method="update",
                 args=[{"visible": vis1}]),
            dict(label="Slots 51–100", method="update",
                 args=[{"visible": vis2}]),
        ])
    ])
fig_overlay.show()

# ── Cat-plots + Welch-t summary table ──────────────────────────────────
metrics = ["UTS_MPa", "Break_Stress_MPa", "Break_Strain_%",
           "Youngs_Modulus_GPa", "Post_Gradient_MPa_perc",
           "Yield_Gradient_MPa_perc"]                       # ← NEW metric added
titles  = ["UTS (MPa)", "Break Stress (MPa)",
           "Break Strain (%)", "Young’s Modulus (GPa)",
           "Post-gradient (MPa / % strain)",
           "Yield gradient (MPa / % strain)"]              # ← NEW title
colors  = {"1-50": rgba(blues[-1]), "51-100": rgba(reds[-1])}

# 6 metrics → 3 rows × 2 cols; add an extra row for the table
cat = make_subplots(
    rows=4, cols=2,
    subplot_titles=titles + ["Welch t-tests"],
    horizontal_spacing=0.12,
    vertical_spacing=0.14,
    specs=[
        [{}, {}],
        [{}, {}],
        [{}, {}],
        [ {}, {"type": "table"}],      # row 4: left blank, table on the right
    ]
)

# ---- add box-plots -----------------------------------------------------
for i, m in enumerate(metrics):
    r, c = divmod(i, 2)           # 0-indexed
    r += 1; c += 1                # make_subplots is 1-indexed
    for cond in ["1-50", "51-100"]:
        cat.add_trace(go.Box(
            y=summary_df[summary_df["Condition"] == cond][m],
            name=cond,
            marker_color=colors[cond],
            boxmean="sd",
            boxpoints="all",
            jitter=0.35,
            pointpos=0,
            showlegend=(i == 0)),               # only show legend once
            row=r, col=c
        )
    cat.update_yaxes(title_text=titles[i], row=r, col=c)
    cat.update_xaxes(showticklabels=False, row=r, col=c)

# ---- Welch-t + effect size table --------------------------------------
def cohen_d(a, b):
    """Unbiased pooled Cohen’s d."""
    n1, n2 = len(a), len(b)
    if n1 < 2 or n2 < 2:
        return np.nan
    s1, s2 = a.std(ddof=1), b.std(ddof=1)
    pooled = math.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))
    return (a.mean() - b.mean()) / pooled if pooled else np.nan

rows_tbl = []
for m in metrics:
    g1 = summary_df[summary_df["Condition"] == "1-50"][m].dropna()
    g2 = summary_df[summary_df["Condition"] == "51-100"][m].dropna()
    if len(g1) and len(g2):
        t, p = ttest_ind(g1, g2, equal_var=False)
        d    = abs(cohen_d(g1, g2))
        sig  = "Yes" if p < 0.05 else "No"
        if   d >= 0.8:  eff = "Large"
        elif d >= 0.5:  eff = "Medium"
        elif d >= 0.2:  eff = "Small"
        else:           eff = "Negligible"
    else:
        t, p, d, sig, eff = (np.nan, np.nan, np.nan, "N/A", "N/A")

    rows_tbl.append([
        titles[metrics.index(m)], f"{t:6.2f}", f"{p:.3g}",
        f"{d:.2f}", sig, eff
    ])

header = ["Metric", "t", "p-value", "|d|", "p < 0.05?", "Effect"]
cat.add_trace(
    go.Table(
        header=dict(values=header, fill_color="#f2f2f2", align="center"),
        cells=dict(values=list(map(list, zip(*rows_tbl))), align="center")
    ),
    row=4, col=2
)

cat.update_layout(
    height=1300, width=1050,
    template="simple_white",
    title="Experiment 002 – metric distributions + Welch t-tests",
    boxmode="group"
)
cat.show()

# ── Export both metrics & stats ────────────────────────────────────────
summary_df.to_excel(ROOT / f"{ROOT.name}_metrics.xlsx", sheet_name="Metrics")
pd.DataFrame(rows_tbl, columns=header).to_excel(
    ROOT / f"{ROOT.name}_stats.xlsx", index=False)

print("[INFO] Metrics exported →", ROOT / f"{ROOT.name}_metrics.xlsx")
print("[INFO] Stats   exported →", ROOT / f"{ROOT.name}_stats.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'Experiment_004\\Dimensional_Data.txt'